# 13 Interfaces, Protocols, and ABCs
Some notes, observations and questions along chapter 13.

"The best approach to understanding a type in Python is knowing the methods it provides—its interface [...]"

- Duck typing: Python’s default approach based on similar methods.
- Goose typing: supported by "abstract base classes" (ABCs); relies on runtime `isinstance()` checks of objects
- Static typing: traditional approach like in C and Java; enforced by external type checkers compliant Type Hints
- Static duck typing: supported by subclasses of `typing.Protocol`

<img src="images/typing_map.png" alt="typing_map" width="700" height="500">

reference: Luciano Ramalho: Fluent Python. Clear, Concise, and Effective Programming, 2022, chapter 13.

### Two Kinds of Protocols
- a protocol is the methods an object implements in order to belong to a certain class
- for instance, a Sequence object needs `__getitem__()` and `__len__()`
- but it doesn't need to be a full sequence, depending on what is the purpose of a class and how it is used
    - a protocol is like an "informal interface"

##### the two kinds are:
1. dynamic typing: duck typing Python always had, defined by convention with informal descriptions (in the Python docs) on what a type needs to implement
2. static typing uses `typing.Protocol` to define one or more methods that a class must implement (or inherit) to satisfy a static type checker

- static protocols are stricter and can be checked by type checkers, dynamic protocols cannot

### Programming Ducks
- the ABCs from `collections.abc` can be used to describe how types look like, but the important thing is is that duck typing works without inheritance
    - also the built-in sequences like `list`, `str`, etc., do not rely on an ABC at all
- an alternative definition of duck typing: "ignoring an object’s actual type, focusing instead on ensuring that the object implements the method names, signatures, and semantics required for its intended use"

#### Sequence types
- if we would inherit from `collections.abc.Sequence`, we would only need to implement `__len__()` and `__getitem__()` which are abstract classes, and get the other methods (`__iter__()`, `__contains__()`, `__reversed__()`, `index()`, `count()`) by inheritance
- `index()`, `count()` are public methods that are supposed to be used by developers
- also, the Python interpreter finds ways to iterate over objects that don't implement `__iter__()` and use the **in** operator on objects without `__contains__()` (in case the class was build without inheriting from `collections.abc.Sequence`)
    - this is the interpreters own implementation in C
    - tries different ways to iterate over anything that remotely resembles a sequence

#### Monkey Patching: Implementing a Protocol at Runtime
- Monkey patching is dynamically changing a module, class, or function at runtime, to add features or fix bugs
- FrenchDeck doesn't support shuffling, because it doesn't have its own `shuffle()` method
- BUT: it doesn't need it's own `shuffle()` method and the pythonic way would be to use `random.shuffle()` on it:

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

from random import shuffle

deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

The error raises, because `shuffle()` operates in place and a `FrenchDeck` object is immutable: it doesn't have a `__setitem__()`.

But here, we can monkey patch:

In [ ]:
def set_card(deck, position, card): # `deck` here is like passing `self`
    deck._cards[position] = card

# at runtime, we assign a new method to the class by setting a function as an attribute named `__setitem__`:
FrenchDeck.__setitem__ = set_card
shuffle(deck)

deck[:5]

[Card(rank='2', suit='hearts'),
 Card(rank='J', suit='spades'),
 Card(rank='10', suit='diamonds'),
 Card(rank='4', suit='spades'),
 Card(rank='J', suit='clubs')]

Note: random.shuffle doesn’t care about the class of the argument, it only needs the object to implement methods from the mutable sequence protocol.

#### Defensive Programming and “Fail Fast”
- means raising runtime errors as soon as possible, for example, rejecting invalid arguments right a the beginning of a function body
- we want to see `TypeError` exceptions in `__init__()` ideally --> easy to find and to fix
- use `try`/`except` if we want to customize the error message (for instance in API)
- we can do this by either calling functions on inputs early in the code that need to be called anyways (such as `iter()` or `list()`) or by using `isinstance()`
    - try to make some argument quark like a duck first, then deal with what happens if it doesn't

### Goose Typing
- abstract base classes can be used to define what other languages call an interface
- but abcs are not checked at runtime, only by type checkers like IDEs or interpreters would implement them
- abcs complement duck typing by providing a way to define interfaces and thus providing virtual subclasses (subclasses without inheritance)
- "What goose typing means is: `isinstance(obj, cls)` is now just fine…as long as cls is an abstract base class"
    - this ensures very abstract, clearly separated functionalities to be checked
    - because something like this is not good to check against:
    
            ```    
            class Artist:
                def draw(self): ...

            class Gunslinger:
                def draw(self): ...

            class Lottery:
                def draw(self): ...
            ``` 
- `register` class method, which lets end-user code “declare” that a certain class becomes a “virtual” subclass of an ABC
    - then it is a virtual subclass

- and sometimes, we don't even need to register, if the class implements certain methods also present in an abc:

In [4]:
class Struggle:
    def __len__(self): return 23

from collections import abc
isinstance(Struggle(), abc.Sized)

True

`Struggle()` is recognised as an instance of `abc.Sized` because it implements `__len__()` AND because `__len__()` is callable without arguments and returns a nonnegative integer denoting an object’s “length”.

"To summarize, goose typing entails:
- Subclassing from ABCs to make it explict that you are implementing a previously defined interface.
- Runtime type checking using ABCs instead of concrete classes as the second argument for isinstance and issubclass."

#### Subclassing an ABC
- the standard way to declare an ABC is to subclass abc.ABC or any other ABC

In [ ]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value): # we need this to make the FrenchDeck2 object shufflable
        self._cards[position] = value

    # we don't need this, but `abc.MutableSequence` requires us to implement this by having an abstract method for this
    def __delitem__(self, position):
        del self._cards[position]

    # we don't need this, but `abc.MutableSequence` requires us to implement this by having an abstract method for this
    def insert(self, position, value):
        self._cards.insert(position, value)

We have to implement `__delitem__` and `insert()` because of inheritance from an abstract base class. But it also provides us with 8 more methods that are not abstract methods. We could override them with our own - more efficient - implementations, but we don't need to.

#### ABCs in the Standard Library
- most abcs are defined in the `collections.abc` module, but there are more in `io`, `numbers` and others
- `collections.abc` has a [table](https://docs.python.org/3/library/collections.abc.html#collections-abstract-base-classes) describing methods and abstract methods their abc implement
- a few of these are:
    - `Iterable` supports iteration with `__iter__`
    - `Container` supports the `in` operator with `__contains__`
    - `Sized` supports `len()` with `
    - `Collection` has no methods of its own, but was added in Python 3.6 to make it easier to subclass from Iterable, Container, and Sized
    - `Sequence`, `Mapping`, `Set` are immutable and each has a mutable subclass
    - `Callable`, `Hashable` are not collections, but still in here; they support type checking objects that must be callable or hashable

#### Defining and Using an ABC
- should be done seldomly, mostly the abc that are already there, are enough
- inheriting from `abc.ABC` is used to create a new abstract base class
- "An abstract method can actually have an implementation. Even if it does, subclasses will still be forced to override it, but they will be able to invoke the abstract method with super(), adding functionality to it instead of implementing from scratch."

In [ ]:
import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.
        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())         # this is very expensive, since it calls inspect()

    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []                          # silly code that can be overridden by subclasses
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(items)

In [7]:
# subclass from Tombola
class Fake(Tombola):
    def pick(self):
        return 13

Fake

__main__.Fake

In [ ]:
# Fake is still an abc, because it doesn't implement its own load() method
f = Fake()

TypeError: Can't instantiate abstract class Fake without an implementation for abstract method 'load'

Implementing real subclasses from our own abc `Tombola`:

In [9]:
import random

class BingoCage(Tombola):

    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        self.pick()

`BingoCage` inherits the the expensive `loaded()` and the silly `inspect()` method from `Tombola` and implements one additional method, `__call__()`.

Alternative subclass:

In [10]:
class LottoBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)    # defensive programming: making sure `iterable` can be converted into 
                                        # a list early (because we want to use `.pop()` on it);
                                        # list(iterable) also creates a copy, which is good practice

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)

    def loaded(self):               # faster than the inherited method from Tombola
        return bool(self._balls)

    def inspect(self):              # faster too and more straightforward
        return tuple(self._balls)

#### A Virtual Subclass of an ABC
- to register a class as a virtual subclass of an ABC, even if it does not inherit from it
- we promise that the class faithfully implements the interface defined in the ABC—and Python will believe us without checking
- will be recognized as such by `issubclass()`, but it does not inherit any methods or attributes from the ABC

In [12]:
from random import randrange

@Tombola.register # there is a register function too (Tombola.register(TomboList)), but here we use the register decorator
class TomboList(list):

    def pick(self):
        if self:  # inherits its boolean behavior from list, and that returns True if the list is not empty
            position = randrange(len(self))
            return self.pop(position)  # pick calls self.pop, inherited from list, passing a random item index
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend  # Tombolist.load is the same as list.extend

    def loaded(self):
        return bool(self)  

    def inspect(self):
        return tuple(self)


In [13]:
issubclass(TomboList, Tombola) # because of the registration

True

In [14]:
t = TomboList(range(100))
isinstance(t, Tombola) # because of the registration

True

In [15]:
TomboList.__mro__ # mro only lists real superclasses, not virtual ones

(__main__.TomboList, list, object)

#### Usage of register in Practice
- many in-build classes (like tuple) are also registering at abcs behind the scenes at import time

#### Structural Typing with ABCs
- sometimes, classes can be recognised as a sublcass (for instance by `issubclass()`) even without inheritin or registering only based on similar methods available
- this is due to a class method called `__subclasskook__()`
- for instance, the `__subclasshook__` for `Sized` checks whether the class argument has an attribute named `__len__`.

### Static Protocols

- introduced in [PEP 544](https://peps.python.org/pep-0544/)

- Question: Are protocols only for type checking or not actually usable like an abstract base class?

In [ ]:
from typing import TypeVar, Protocol

T = TypeVar('T') # TypeVar() acts as a placeholder for an unspecified type; T is a type variable

class Repeatable(Protocol):
    def __mul__(self: T, repeat_count: int) -> T: ...  # let type checker know that the return value is the same type as self

RT = TypeVar('RT', bound=Repeatable) # bound means that RT can only represent types that fulfill the requirements of `Repeatable`

def double(x: RT) -> RT: # since RT is bound to `Repeatable`, `x` must support the `__mul__` operation as defined in `Repeatable`
    return x * 2

`T = TypeVar('T')` is a way to explain to the type checker that the return type is the same as the type of the argument `self`, see this [link](https://dev.to/decorator_factory/typevars-explained-hmo) for an explanation.

`RT = TypeVar('RT', bound=Repeatable)` fulfills the same purpose as `T` above, and the `bound` means that `RT` can only represent types that fulfill the requirements of `Repeatable`. And since `Repeatable` is a `Protocol` specifying that an object must have a `__mul__` method that returns the same type as itself, any type assigned to RT must support this multiplication operation (`*`).

**Use case**: Enforce that any object passed to double must support the `__mul__` method.
"The nominal type of the actual argument x given to double is irrelevant as long as it quacks—that is, as long as it implements `__mul__`."

#### Runtime Checkable Static Protocols
- using Protocol class enables static type checking, but we can use the `@runtime_checkable` decorator to make that protocol support `isinstance`/`issubclass` checks at runtime
- if I understand correctly, then some or all types defined in `typing` are at the same time a `Protocol`
    - for instance `typing.SupportsComplex` is a `Protocol`; it's only special code is this:

In [21]:
from abc import abstractmethod

    @abstractmethod
    def __complex__(self) -> complex:
        pass

IndentationError: unexpected indent (4011459289.py, line 3)

- with the `@runtime_checkable` decorator, the `SupportsComplex` `Protocol` class can be used at runtime
- example:

In [22]:
from typing import SupportsComplex
import numpy as np

c64 = np.complex64(3+4j) # complex64 is one of five complex number types provided by NumPy
isinstance(c64, complex) #None of the NumPy complex types subclass the built-in complex.

False

In [23]:
isinstance(c64, SupportsComplex) # But NumPy’s complex types implement __complex__, so they comply with the SupportsComplex protocol

True

In [24]:
c = complex(c64) # therefore, we can create built-in complex objects from them (which are also complex)
c

(3+4j)

In [ ]:
isinstance(c, SupportsComplex) # the complex built-in type simplement __complex__ itself 
# (in Fluent Python that was not so, but maybe this has changed in the meantime)

True

In [26]:
complex(c) # complex(c) works fine if c is a complex

(3+4j)

#### Duck Typing is your Friend
--> it's easier to ask for forgiveness than permission

`Protocol` way:
```
if isinstance(o, (complex, SupportsComplex)):
    # do something that requires `o` to be convertible to complex
else:
    raise TypeError('o must be convertible to complex')
```

Goose typing way:
```
if isinstance(o, numbers.Complex):
    # do something with `o`, an instance of `Complex`
else:
    raise TypeError('o must be an instance of Complex')
```

Duck typing way:
```
try:
    c = complex(o)
except TypeError as exc:
    raise TypeError('o must be convertible to complex') from exc
```

or simply `c = complex(o)` in case it's our own code (not API for others to use) and we don't need to raise a new error message

#### Limitations of Runtime Protocol Checks
- type checking at runtime (using `isinstance()` or `issubclass()`) only checks if a method is available, not if it returns the correct type (type hints are not included)
    - these checks only look at the presence or absence of methods, without checking their signatures

#### Supporting and Designing a Static Protocol
- making a custom stating Protocol:

In [27]:
from typing import Protocol, runtime_checkable, Any

@runtime_checkable
class RandomPicker(Protocol):
    def pick(self) -> Any: ...
    """Some docstring like in abstract methods."""

- `SimplePicker` class uses this `RandomPicker` Protocol as a blueprint:

In [29]:
import random
from typing import Any, Iterable, TYPE_CHECKING

class SimplePicker: # SimplePicker implements RandomPicker—but it does not subclass it. This is static duck typing in action
    def __init__(self, items: Iterable) -> None:
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self) -> Any:  # Any is the default return type, so this annotation is not strictly necessary, 
                            # but it does make it more clear that we are implementing the RandomPicker protocol
        return self._items.pop()

- Question: but how does `SimplePicker` know that `RandomPicker` is supposed to be its `Protocol`?

- this is how we could test if it works:

In [30]:
def test_isinstance() -> None:
    popper: RandomPicker = SimplePicker([1]) # RandomPicker is a type hint, tested only by mypy
    assert isinstance(popper, RandomPicker) # proves that an instance of SimplePicker is also an instance of RandomPicker

def test_item_type() -> None:
    """This test invokes the pick method from a SimplePicker, verifies that it returns
    one of the items given to SimplePicker, and then does static and runtime checks on
    the returned item."""
    items = [1, 2]
    popper = SimplePicker(items)
    item = popper.pick()
    assert item in items
    
    # this is for static type checking in mypy
    if TYPE_CHECKING:   # if block is protected by typing.TYPE_CHECKING, which is only True 
                        # in the eyes of a static type checker, but is False at runtime
        reveal_type(item) # This line generates a note in the Mypy output

    # this is for runtime type checking    
    assert isinstance(item, int)

#### Best Practices for Protocol Design
- minimalistic protocols ideally only supporting one method
- talks about naming conventions

#### Extending a Protocol
- a few caveats:

In [31]:
from typing import Protocol, runtime_checkable

@runtime_checkable # we must apply the decorator again; its behavior is not inherited
class LoadableRandomPicker(RandomPicker, Protocol): # very protocol must explicitly name 
                                                    # `typing.Protocol` as one of its base classes in addition 
                                                    # to the protocol we are extending; this is different from 
                                                    # how inheritance usually works
    def load(self, Iterable) -> None: ...

#### The numbers ABCs and Numeric Protocols


- runtime type checking is well defined in Python for numeric types
    - for instance, checking for an integer: use `isinstance(x, numbers.Integral)` to accept `int`, `bool` (which subclasses int) or other integer types that are provided by external libraries that register their types as virtual subclasses of the numbers ABCs
        - numpy has 21 integer types, such as `int32`
        - but remember: `isinstance()` might return misleading results, since it only checks for methods implemented, not their signatures nor type hints in the signature
        - and the other way around might also give misleading results: e.i. `numpy.uint8`, doesnt have a` __complex__` method (and would thus fail the `isinstance(x, SupportsComplex)` check), but could easily implement it (if we do it -- via monkeypatching???)
- it's not so easy for static type checking: currently it is recommenden that type checkers hardcode the subtype relationships among built-in `complex`, `float`, and `int`